~/github/cm4ai_codefest2025
│
├── data/                # contains red/, yellow/, blue/, green/
├── analysis/            # where you want results saved
├── HPA-Cell-Segmentation/  # git clone lives here
└── HPA Cell Segmentation Script.ipynb

In [ ]:
# ==========================================
# HPA Cell Segmentation - Full FOV to Masks
# Project root: ~/github/cm4ai_codefest2025
# ==========================================

# Change directory to project root
%cd ~/github/cm4ai_codefest2025

from pathlib import Path
import numpy as np
from imageio import imwrite
from hpacellseg.cellsegmentator import CellSegmentator
from hpacellseg.utils import label_cell

# =====================
# PATHS
# =====================
DATA_ROOT    = Path("./data")
ANALYSIS_DIR = Path("./analysis/segmentation_results2")
ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

NUC_MODEL  = "./HPA-Cell-Segmentation/nuclei-model.pth"
CELL_MODEL = "./HPA-Cell-Segmentation/cell-model.pth"


In [ ]:

# =====================
# SEGMENTATOR
# =====================
segmentator = CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="mps",             # "cuda", "mps", or "cpu"
    padding=True,
    multi_channel_model=True
)

# =====================
# LOAD DATA
# =====================
blue_imgs   = sorted((DATA_ROOT / "blue").glob("*.jpg"))
red_imgs    = sorted((DATA_ROOT / "red").glob("*.jpg"))
yellow_imgs = sorted((DATA_ROOT / "yellow").glob("*.jpg"))
green_imgs  = sorted((DATA_ROOT / "green").glob("*.jpg"))  # optional 4th channel

# Make sure filenames line up (same order)
assert len(blue_imgs) == len(red_imgs) == len(yellow_imgs) == len(green_imgs), "Channel mismatch"

fovs = list(zip(red_imgs, yellow_imgs, green_imgs, blue_imgs))

# =====================
# RUN SEGMENTATION
# =====================
for i, (red, yellow, green, blue) in enumerate(fovs):
    base = blue.stem.replace("_blue","")
    print(f"Processing {base} ({i+1}/{len(fovs)})")

    nuc_seg  = segmentator.pred_nuclei([str(blue)])[0]
    cell_seg = segmentator.pred_cells([[str(red)], [str(yellow)], [str(green)], [str(blue)]])[0]

    nuclei_mask, cell_mask = label_cell(nuc_seg, cell_seg)

    # save masks
    imwrite(ANALYSIS_DIR / f"{base}_nuclei_mask.png", nuclei_mask.astype(np.uint16))
    imwrite(ANALYSIS_DIR / f"{base}_cell_mask.png", cell_mask.astype(np.uint16))

    np.save(ANALYSIS_DIR / f"{base}_nuclei_mask.npy", nuclei_mask)
    np.save(ANALYSIS_DIR / f"{base}_cell_mask.npy", cell_mask)

print("Segmentation complete. Results saved to", ANALYSIS_DIR)
